In [3]:
# All import function here
from serpapi import GoogleSearch
from gsearch.googlesearch import search
from pprint import pprint
from search_engine_parser.core.engines.google import Search as GoogleSearch
import pprint
import nest_asyncio
import pandas as pd
from datetime import datetime, timedelta

nest_asyncio.apply()
gsearch = GoogleSearch()

In [22]:
gsearch = GoogleSearch()
search_args = ('tesla news stock after:2019-03-08 before:2019-03-10', 1)
gresults = gsearch.search(*search_args)
# print(gresults['links'])
dir(gresults)

['_SearchResult__repr_',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'append',
 'keys',
 'results']

In [24]:
gresults.keys()

dict_keys(['titles', 'links', 'descriptions'])

In [25]:
gresults['description']

['TSLA Stock Price | Tesla Inc(NASDAQ:TSLA) stock summaries, charts and news.',
 '$1,000, 5 Years Later: How Much Would Tesla Stock Be Worth?www.benzinga.com > general > education > 1-000-5-years-later-how-much...',
 'Warren Buffett Just Invested $4.1 Billion Into Big Oil - CleanTechnicacleantechnica.com > 2021/02/16 > warren-buffett-just-invested-4-1-billion-...',
 "Tesla's dirty little secret: Its net profit doesn't come from selling carswww.mercurynews.com > 2021/02/01 > teslas-dirty-little-secret-its-net-prof...",
 'Why Tesla Stock Fell Sharply on Tuesday | The Motley Foolwww.fool.com > investing > 2021/03/16 > why-tesla-stock-fell-sharply-on-...',
 'Mar 9, 2019 · Reading between the lines of the Tesla news circus reveals a very different story than the one Elon Musk puts out.',
 'Mar 9, 2019 · Check out the latest earnings call transcript for Tesla. ... In early 2017, SolarCity -- now acquired by Tesla for $2.9 billion in stock and the\xa0...',
 'Mar 10, 2019 · Last month, we anno

In [26]:
gresults['titles']

['Tesla Inc(NASDAQ:TSLA) Quote, Price, Charts & News | CMLViz.com',
 'Tesla investors that bought and held on through a volatile five-year period turned a massive profit. In fact, $1,000 worth of Tesla stock bought in 2016 would be worth about $22,700 today.',
 'Buffett has never invested in Tesla but has invested $1.57 billion in GM. He said he would not invest in Tesla in an interview with Yahoo! Finance.',
 "It's a lucrative business for Tesla — bringing in $3.3 billion over the course of the last five years, nearly half of that in 2020 alone. The $1.6 billion in regulatory credits it received last year far outweighed Tesla's net income of $721 million — meaning Tesla would have otherwise posted a net loss in 2020.",
 'Shares of Tesla (NASDAQ:TSLA) took a hit on Tuesday. The electric car maker slid by as much as 5.2%, and ended the trading day down by 4.4%. That decline was likely primarily due to a broad pullback in the prices of many growth stocks toward the end of the session.',


In [34]:
csv = pd.read_csv("data/tesla_3_months.csv")

In [35]:
print(csv)

          Date        Open        High         Low       Close   Adj Close  \
0   2021-01-25  855.000000  900.400024  838.820007  880.799988  880.799988   
1   2021-01-26  891.380005  895.900024  871.599976  883.090027  883.090027   
2   2021-01-27  870.349976  891.500000  858.659973  864.159973  864.159973   
3   2021-01-28  820.000000  848.000000  801.000000  835.429993  835.429993   
4   2021-01-29  830.000000  842.409973  780.099976  793.530029  793.530029   
..         ...         ...         ...         ...         ...         ...   
58  2021-04-19  719.599976  725.400024  691.799988  714.630005  714.630005   
59  2021-04-20  717.419983  737.250000  710.690002  718.989990  718.989990   
60  2021-04-21  704.770020  744.840027  698.000000  744.119995  744.119995   
61  2021-04-22  741.500000  753.770020  718.039978  719.690002  719.690002   
62  2021-04-23  719.799988  737.359985  715.460022  729.400024  729.400024   

      Volume  
0   41173400  
1   23131600  
2   27334000  
3  

In [38]:
csv['date']


KeyError: 'date'

In [47]:
class CreateStockNewsPriceDatabaseClass:
    def __init__(self):
        self.path_to_csv_for_stock_price = "data/tesla_3_months.csv"
        self.write_csv_to_file = "data/tesla_3_months_news_stock_price_data"
        self.days_window = 3
        self.stock_name = 'tesla'
        self.time_window_for_stock_calculation = 1
        
    def read_csv_stock_data(self):
        csv = pd.read_csv(self.path_to_csv_for_stock_price)
        return csv
    
    def return_title_description(self, end_ds, stock_company_name, days_window=3):
        end_date = datetime.strptime(end_ds, '%Y-%m-%d') 
        end_date_string_format = end_date.strftime("%Y-%m-%d")

        start_date = end_date - timedelta(days=days_window)
        start_date_string_format = start_date.strftime("%Y-%m-%d")
        search_args = ("{stock_company_name} news stock after:{start_date_string_format} before:{end_date_string_format}".format(stock_company_name=stock_company_name, start_date_string_format=start_date_string_format, end_date_string_format=end_date_string_format), 1)
        gresults = gsearch.search(*search_args)
        title = ';'.join(title for title in gresults['titles'])
        description = ';'.join(description for description in gresults['description'])
        return end_date_string_format, title, description
    
    def map_stock_news_with_price(self):
        stock_price_csv = self.read_csv_stock_data()
        titles = []
        descriptions = []
        price_delta_of_stocks = []
        for index, row in stock_price_csv.iterrows():
            ds, title, description = self.return_title_description(end_ds=row['Date'], stock_company_name='tesla', days_window=3)
            price_delta_of_stock = stock_price_csv['Close'][index+self.time_window_for_stock_calculation] - row['Open'] if index + self.time_window_for_stock_calculation < stock_price_csv.shape[0] else 0
            
            titles.append(title)
            descriptions.append(description)
            price_delta_of_stocks.append(price_delta_of_stock)
            
        stock_price_csv['title'] = titles
        stock_price_csv['description'] = descriptions
        stock_price_csv['price_delta_of_stock'] = price_delta_of_stocks
        self.write_to_csv(stock_price_csv)
    
    def write_to_csv(self, object_to_write):
        self.write_csv_to_file = self.write_csv_to_file + "_" + str(self.days_window) + "_days_" + str(self.time_window_for_stock_calculation) + "_delta.csv"
        object_to_write.to_csv(self.write_csv_to_file)


In [48]:
create_stock_news_price_database_class=CreateStockNewsPriceDatabaseClass()
# create_stock_news_price_database_class.return_title_description(end_ds='2019-03-10', stock_company_name='apple', days_window=3)
create_stock_news_price_database_class.map_stock_news_with_price()